In [1]:
# Description: Plot figure 8 (difference between composites of high and low SAM indices)
#              -(a): Map of the difference between the ensemble averages of Tbot from
#                    the 90th and the 10th percentiles of the annually-averaged SAM index.
#              -(b): Same as panel (a), but for the Niño 3.4 index.
#
# Here, "Tbot" denotes the model potential temperature (referenced to 0 dbar)
# at the grid cell closest to the bottom or closest to 1000 m depth, whichever
# is shallower.
#
# Author:      André Palóczy
# E-mail:      paloczy@gmail.com
# Date:        January/2018

import sys
sys.path.append('../../misc')
sys.path.append('../../data_reproduce_figs')
sys.path.append('/home/andre/Dropbox/python-modules/pygamman')

# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

def bmap_antarctica(ax, projection='spstere', bbox=[-180, 180, -77, -59], resolution='i'):
    """
    Full Antartica basemap (Polar Stereographic Projection).
    """
    kwm = dict(projection=projection, resolution=resolution, ax=ax)
    if projection=='spstere':
        kwm.update(dict(boundinglat=-60, lon_0=60))
    else:
        lol, lor, lal, lar = bbox
        kwm.update(dict(llcrnrlon=lol, urcrnrlon=lor,
        llcrnrlat=lal, urcrnrlat=lar, fix_aspect=False))

    m = Basemap(**kwm)

    m.drawmapboundary(zorder=-9999)
    m.drawcoastlines(zorder=10, linewidth=0.5)
    if projection=='spstere':
        m.fillcontinents(color='0.9', zorder=9)
        m.drawmeridians(np.arange(-180, 180, 45), linewidth=0.15, labels=[0, 1, 0, 1], zorder=12)
    else:
        m.drawmeridians(np.arange(-180, 180, 45), linewidth=0.15, labels=[0, 0, 0, 0], zorder=12)
    m.drawparallels(np.arange(-90, -50, 5), linewidth=0.15, labels=[0, 0, 0, 0], zorder=12)

    return m

In [2]:
plt.close('all')

projection = 'spstere'
figsize = (8.5, 11)
fmt = 'png'
kwsavefig = dict(fmt=fmt, dpi=150, bbox_inches='tight', pad_inches=0.0)

SAM_percentile = 10
NINO34_percentile = 10

head = '../../data_reproduce_figs/'
fpt = head + 'yearly_PT.npz'
fname_isobs = head + 'isobaths.nc'

In [ ]:
# Load isobaths.
misob = 1000
iisob, oisob = 800, 2500
ncxm = Dataset(fname_isobs)
xmisob = ncxm["%d m isobath (U-points)"%misob]['xiso'][:]
ymisob = ncxm["%d m isobath (U-points)"%misob]['yiso'][:]
xiisob = ncxm["%d m isobath (U-points)"%iisob]['xiso'][:]
yiisob = ncxm["%d m isobath (U-points)"%iisob]['yiso'][:]
xoisob = ncxm["%d m isobath (U-points)"%oisob]['xiso'][:]
yoisob = ncxm["%d m isobath (U-points)"%oisob]['yiso'][:]

In [ ]:
# Load model potential temperature fields.
d = np.load(fpt)
xpt = d['lon']
ypt = d['lat']
zlevpt = 350#d['zlev']
tmod = d['t'].tolist()
pt = d['pt'] # [degC].

In [3]:
# Calculate High SAM/Niño 3.4 and low SAM/Niño 3.4 ensemble averages.
tmod = np.array(tmod)

# SAM.
SAM_percentile_upper = 100 - SAM_percentile
lopercSAM = np.percentile(SAM, SAM_percentile)
hipercSAM = np.percentile(SAM, SAM_percentile_upper)
floSAM = SAM<=lopercSAM
fhiSAM = SAM>=hipercSAM
floSAM = floSAM.values
fhiSAM = fhiSAM.values

SAMlo = SAM[floSAM]
SAMhi = SAM[fhiSAM]
yearslo, yearshi = SAMlo['time.year'], SAMhi['time.year']
loSAM = SAMlo.mean().values.flatten()[0]
hiSAM = SAMhi.mean().values.flatten()[0]
print("Mean high SAM = %.1f"%hiSAM)
print("Mean low SAM = %.1f"%loSAM)
yearsCORE = slp['TIME.year'].values.tolist()
floCORE = np.array([yearCORE in yearslo.values for yearCORE in yearsCORE])
fhiCORE = np.array([yearCORE in yearshi.values for yearCORE in yearsCORE])

# Niño 3.4..
NINO34_percentile_upper = 100 - NINO34_percentile
lopercNINO34 = np.percentile(NINO34, NINO34_percentile)
hipercNINO34 = np.percentile(NINO34, NINO34_percentile_upper)
floNINO34 = NINO34<=lopercNINO34
fhiNINO34 = NINO34>=hipercNINO34
floNINO34 = floNINO34.values
fhiNINO34 = fhiNINO34.values

NINO34lo = NINO34[floNINO34]
NINO34hi = NINO34[fhiNINO34]
yearslo_nino34, yearshi_nino34 = NINO34lo['time.year'], NINO34hi['time.year']
loNINO34 = NINO34lo.mean().values.flatten()[0]
hiNINO34 = NINO34hi.mean().values.flatten()[0]
print("Mean high Niño 3.4 = %.1f"%hiNINO34)
print("Mean low Niño 3.4 = %.1f"%loNINO34)

NameError: name 'tmod' is not defined

In [ ]:

loSAMpt = np.nanmean(pt[floMOD, ...], axis=0)
hiSAMpt = np.nanmean(pt[fhiMOD, ...], axis=0)
loNINO34pt = np.nanmean(pt[floMOD_NINO34, ...], axis=0)
hiNINO34pt = np.nanmean(pt[fhiMOD_NINO34, ...], axis=0)